In [1]:
'''

從MySQL取得product table
以及 userID，用來創新文件

'''
import pymysql
import pprint

# open connection(host, user, password, database)
host = "uuboyscymysql.clrhltpp3icl.ap-northeast-1.rds.amazonaws.com"
port = int(3306)
user = "food"
password = "food"
db = "food"

conn = pymysql.connect(
    host = host,
    port = port,
    user = user,
    password = password,
    db = db,
    charset='utf8',    
    cursorclass=pymysql.cursors.DictCursor )

print('Connected successfully!')

cursor = conn.cursor()
cursor.execute("SELECT `productName`, `productUnitPrice` from productTable;")

product_dict_list = cursor.fetchall()

print(product_dict_list)

cursor.execute("SELECT `userID` FROM membersInfo;")

userID_dist_list = cursor.fetchall()
userID_list = []
transaction_status_dict = {}

for i in userID_dist_list:
    userID_list.append(i['userID'])
print(userID_list)

for i in userID_list: # i for userID, j for product name
    transaction_status_dict[i] = {}
    for num, j in enumerate(product_dict_list):
        #print(j['productName'])
        transaction_status_dict[i][j['productName']] = [0, product_dict_list[num]['productUnitPrice'], 0, num+1] # quantity, unit price, subtotal
    
print('\n=====\n')
pprint.pprint(transaction_status_dict)

conn.close()

Connected successfully!
[{'productName': 'Salt', 'productUnitPrice': 40}, {'productName': 'Sauce', 'productUnitPrice': 50}, {'productName': 'Egg', 'productUnitPrice': 30}, {'productName': 'Chicken', 'productUnitPrice': 150}, {'productName': 'Pork', 'productUnitPrice': 170}, {'productName': 'Flour', 'productUnitPrice': 75}, {'productName': 'OliveOil', 'productUnitPrice': 120}, {'productName': 'Cream', 'productUnitPrice': 100}]
['U0fe5e2ddf71a138b6e21d4a970592127', 'U19a0dc2519f15c625ea764802cd62202', 'U1e91bf35eb5315bc17da624e46a65c51', 'U26c1d062b5768c0ab8721698973d67b1', 'U2dd03f0598a942d9050eea3c4676f85d', 'U3635a6a27c2907e7e6f1cf2d8dd41a0c', 'U400b95f5a7d6cb62ef336f96b26de6dd', 'U57dae482fd0d30eddb584a0e3e69d7fd', 'U5dc989d65f1897bd44fcbd415b47e164', 'U6d22d8292171bb2cfd96d2b9ef5383a9', 'U7101f9f716cfb37a89a2a09207636b17', 'U7173dc299af6ac3fe7697a75c2a280ad', 'U7cb420838034721c73198cf46a9fb195', 'U7d56a120b59383802416bde5192f0adb', 'U8b575a120d53d40ad98617d52a1ba9b4', 'U96ed7dbbe707

In [2]:
for i in transaction_status_dict['Ue5148dd83b43580ee1b47aa93a371244']:
    print(i)

Salt
Sauce
Egg
Chicken
Pork
Flour
OliveOil
Cream


In [ ]:
from kafka import KafkaConsumer, TopicPartition, KafkaClient
import sys
import json
import requests

secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


if __name__ == "__main__":
    # 步驟1.設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # Kafka集群在那裡?
        bootstrap_servers=["34.80.39.244:9092"],
        # ConsumerGroup的名稱
        group_id="chatbot_consumer",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
         key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        value_deserializer=bytes.decode,
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="earliest",
    )

####################################################


    # 步驟2.指定想要訂閱訊息的topic名稱
    topic_name = 'U7173dc299af6ac3fe7697a75c2a280ad'
    item_num_for_each_userID = {}
    for i in userID_list:
        with open('./transaction_status/%s'%(i), 'w', encoding='utf-8') as f:
            outStr = """
            <tr class="success">
            <td>null</td>
            <td>null</td>
            <td>null</td>
            <td>$ null</td>
            <td>$ null</td>
            </tr>
            """
            f.write(outStr)
        item_num_for_each_userID[i] = 1 # 判斷row顏色、是否刪除第一筆null data
        
########################################
        
    # 步驟3.讓Consumer向Kafka集群訂閱指定的topic
    #consumer.subscribe(topics=topic_name)
    consumer.subscribe(pattern='U[0-9a-z]{32}')
    
    # 表格顏色 list
    tcolor = ["success", "warning"]
    num_color = 0
    #new_transaction = 1
     
    outStr = """"""

    # 步驟4.持續的拉取Kafka有進來的訊息
    try:
        print("Start listen incoming messages ...")
        # 持續監控是否有新的record進來
        for record in consumer:
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
            
            wmsg = "topic=%s, partition=%s, offset=%s : (key=%s, value=%s) , timestamp=%s" % (topic, partition,
                                                                              offset, msgKey, msgValue, timestamp)
            # 將消費紀錄寫入檔案
            if not msgKey == 'end':
                if item_num_for_each_userID[topic] == 1:
                    with open('./transaction_status/%s'%(topic), 'w', encoding='utf-8') as f:
                        f.write('')
                
                # 製作dataframe
                outStr = """"""
                col = 0
                total = 0
                for product_status in transaction_status_dict[topic]:
                    if transaction_status_dict[topic][product_status][3] == int(msgKey):
                        transaction_status_dict[topic][product_status][0] = int(msgValue)
                        product_qty = transaction_status_dict[topic][product_status][0]
                        product_unit_price = transaction_status_dict[topic][product_status][1]
                        transaction_status_dict[topic][product_status][2] = product_qty * product_unit_price
                        subtotal = transaction_status_dict[topic][product_status][2]
                    else:
                        product_qty = transaction_status_dict[topic][product_status][0]
                        product_unit_price = transaction_status_dict[topic][product_status][1]
                        subtotal = transaction_status_dict[topic][product_status][2]
                    total += subtotal
                    
                    if not product_qty == 0:
                        outStr += """
                            <tr class=%s>
                            <td>%s</td>
                            <td>%s</td>
                            <td>%s</td>
                            <td>$ %s</td>
                            <td>$ %s</td>
                            </tr>
                        """%(tcolor[col%2], col+1,
                                product_status, 
                                product_qty,
                                product_unit_price,
                                subtotal
                            )
                        col += 1
                outStr += """
                    <thead>
                        <tr class="danger">
                        <th>Total</th>
                        <th>-</th>
                        <th>-</th>
                        <th>-</th>
                        <th>$ %s</th>
                        </tr>
                    </thead>
                    """%(total)
                with open('./transaction_status/%s'%(topic), 'w', encoding='utf-8') as f:
                    f.write(outStr)
                
                '''with open('./transaction_status/%s'%(topic), 'a', encoding='utf-8') as f:
                    outStr = """
                        <tr class=%s>
                        <td>%s</td>
                        <td>%s</td>
                        <td>%s</td>
                        <td>$ %s</td>
                        <td>$ null</td>
                        </tr>
                    """%(tcolor[item_num_for_each_userID[topic]%2], item_num_for_each_userID[topic], 
                                                                    product_dict_list[int(msgKey)-1]['productName'],
                                                                    str(msgValue),
                                                                    product_dict_list[int(msgKey)-1]['productUnitPrice']
                        )
                    f.write(outStr)'''
                item_num_for_each_userID[topic] += 1
                
            if msgKey == 'end':
                with open('./transaction_status/%s'%(topic_name), 'w', encoding='utf-8') as f:
                    outStr = """
                        <tr class="success">
                        <td>null</td>
                        <td>null</td>
                        <td>null</td>
                        <td>$ null</td>
                        <td>$ null</td>
                        </tr>
                    """
                    f.write(outStr)
                # 將該次交易紀錄上傳MySQL
                conn = pymysql.connect(
                        host = host,
                        port = port,
                        user = user,
                        password = password,
                        db = db,
                        charset='utf8',    
                        cursorclass=pymysql.cursors.DictCursor )
                cursor = conn.cursor()
                
                for product_status in transaction_status_dict[topic]:
                    if not transaction_status_dict[topic][product_status][0] == 0:
                        cursor.execute("INSERT INTO transaction SET orderID=%s, productID=%s, userID=%s, productQTY=%s, datetime=FROM_UNIXTIME(%s)",
                                          (int(timestamp/1000),
                                          transaction_status_dict[topic][product_status][3],
                                          topic,
                                          transaction_status_dict[topic][product_status][0],
                                          int(timestamp/1000)
                                          )
                                      )
                        '''sql += "(%s,%s,'%s',%s,FROM_UNIXTIME(%s));"%(
                                                                    int(timestamp/1000),
                                                                    msgKey,
                                                                    topic,
                                                                    transaction_status_dict[topic][product_status][0],
                                                                    int(timestamp/1000)
                                                                    )
                        '''
                        #print('-----%s'%(sql))
                conn.commit()
                conn.close()
                requests.get('https://%s/transaction_commit/%s'%(server_url, topic))
                print('\nFrom : %s\nTransaction Completed!\n'%(topic))
                
                item_num_for_each_userID[topic] = 1
                
            # 秀出metadata與msgKey & msgValue訊息
            print(wmsg)
            
    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        print('Bye!')
        conn.close()
        consumer.close()

Start listen incoming messages ...
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513272 : (key=1, value=1) , timestamp=1551200868757
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513273 : (key=2, value=1) , timestamp=1551200873758
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513274 : (key=1, value=2) , timestamp=1551200878758
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513275 : (key=7, value=2) , timestamp=1551200883759
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513276 : (key=6, value=2) , timestamp=1551200888760
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513277 : (key=6, value=3) , timestamp=1551200893761
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513278 : (key=6, value=2) , timestamp=1551200898762

From : U7173dc299af6ac3fe7697a75c2a280ad
Transaction Completed!

topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513279 : (key=end, value=1) , timestamp=1551200

topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513321 : (key=4, value=2) , timestamp=1551207560436
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513322 : (key=5, value=1) , timestamp=1551207565437
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513323 : (key=7, value=1) , timestamp=1551207570438
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513324 : (key=6, value=1) , timestamp=1551207575439

From : U7173dc299af6ac3fe7697a75c2a280ad
Transaction Completed!

topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513325 : (key=end, value=1) , timestamp=1551207580440
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513326 : (key=1, value=1) , timestamp=1551209980440
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513327 : (key=2, value=1) , timestamp=1551209985441
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513328 : (key=1, value=2) , timestamp=1551209990441
topic=U7173dc299af6ac3fe7697

topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513381 : (key=2, value=1) , timestamp=1551217510496
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513382 : (key=1, value=2) , timestamp=1551217515496
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513383 : (key=4, value=1) , timestamp=1551217520497
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513384 : (key=2, value=0) , timestamp=1551217525498
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513385 : (key=1, value=1) , timestamp=1551217530499
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513386 : (key=4, value=2) , timestamp=1551217535500
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513387 : (key=5, value=1) , timestamp=1551217540501
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513388 : (key=7, value=1) , timestamp=1551217545502
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513389 : (key=6, value=1) , timesta

topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513441 : (key=6, value=2) , timestamp=1551225065556
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513442 : (key=6, value=3) , timestamp=1551225070557
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513443 : (key=6, value=2) , timestamp=1551225075558

From : U7173dc299af6ac3fe7697a75c2a280ad
Transaction Completed!

topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513444 : (key=end, value=1) , timestamp=1551225080559
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513445 : (key=1, value=1) , timestamp=1551227480559
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513446 : (key=2, value=1) , timestamp=1551227485560
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513447 : (key=1, value=2) , timestamp=1551227490560
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513448 : (key=4, value=1) , timestamp=1551227495561
topic=U7173dc299af6ac3fe7697